In [1]:
import GEOparse
import pandas as pd
import scanpy as sc

In [1]:
import scanpy as sc
import os
from glob import glob

In [32]:
import os
import scanpy as sc
from scipy import io
import pandas as pd

# Define the directory where the files are stored
data_dir = "/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/trem2/GSE183068_RAW"

# Initialize an empty list to store individual AnnData objects
adata_list = []

# Loop through each sample by identifying unique sample prefixes (like `GSM5550450_Gan_1`)
for sample_prefix in set(f.split('.')[0] for f in os.listdir(data_dir) if f.endswith('.gz')):
    # Construct file paths for matrix, barcodes, and features for each sample
    matrix_path = os.path.join(data_dir, f"{sample_prefix}.matrix.mtx.gz")
    barcodes_path = os.path.join(data_dir, f"{sample_prefix}.barcodes.tsv.gz")
    features_path = os.path.join(data_dir, f"{sample_prefix}.features.tsv.gz")
    
    # Load matrix, barcodes, and features
    matrix = io.mmread(matrix_path).tocsr()  # Read the matrix.mtx file as a sparse matrix
    barcodes = pd.read_csv(barcodes_path, header=None).values.flatten()  # Read barcodes as cell identifiers
    features = pd.read_csv(features_path, header=None, sep = '\t')[1].values  # Read features as gene names
    
    # Create AnnData object for the sample
    adata = sc.AnnData(matrix.T)
    adata.obs['cell_ids'] = barcodes
    adata.var['gene_symbols'] = features
    adata.obs['sample'] = sample_prefix  # Track the sample name
    
    # Append to the list
    adata_list.append(adata)

# Concatenate all AnnData objects into one
merged_adata = adata_list[0].concatenate(*adata_list[1:], join='outer', batch_key='sample')

# Save the merged AnnData to a .h5ad file
# merged_adata.write_h5ad("merged_data.h5ad")

/tmp/ipykernel_4080470/4011898620.py:34: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  merged_adata = adata_list[0].concatenate(*adata_list[1:], join='outer', batch_key='sample')


In [33]:
merged_adata

AnnData object with n_obs × n_vars = 386118 × 33538
    obs: 'cell_ids', 'sample'
    var: 'gene_symbols'

In [43]:
merged_adata.obs_names = merged_adata.obs.cell_ids
merged_adata.obs.cell_ids.index.name = None

merged_adata.var_names = merged_adata.var.gene_symbols
merged_adata.var.index.name = None

/home/icb/zihe.zheng/miniconda3/envs/scanpy2/lib/python3.9/site-packages/anndata/_core/anndata.py:841: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    ['AAACCCAAGCTGTTCA-1', 'AAACCCACAGTTCTAG-1', 'AAACCCAGTAGGAGGG-1', 'AAACCCAGTATGCAAA-1', 'AAACCCAGTCGGTAAG-1']

    Inferred to be: categorical

  names = self._prep_dim_index(names, "obs")


In [41]:
merged_adata.var

,gene_symbols
MIR1302-2HG,MIR1302-2HG
FAM138A,FAM138A
OR4F5,OR4F5
AL627309.1,AL627309.1
AL627309.3,AL627309.3
...,...
AC233755.2,AC233755.2
AC233755.1,AC233755.1
AC240274.1,AC240274.1
AC213203.1,AC213203.1


In [46]:
merged_adata.write_h5ad("/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/trem2/human.h5ad")